# TPC3: Processador de Pessoas listadas nos Róis de Confessados

(publicado em **2023.02.28**)

Construa agora um ou vários programas Python
para  processar o texto 'processos.txt' (procurar o ficheiro no Bb) com o intuito de
calcular frequências de alguns elementos (a ideia é utilizar arrays associativos, dicionários em Python,
para o efeito) conforme solicitado a seguir:

* a) Calcula a frequência de processos por ano (primeiro elemento da data);

* b) Calcula a frequência de nomes próprios (o primeiro em cada nome) e apelidos (o ultimo em cada nome) por séculos e apresenta os 5 mais usados;

* c) Calcula a frequência dos vários tipos de relação: irmão, sobrinho, etc.;

* d) Converta os 20 primeiros registos num novo ficheiro de output mas em formato **Json**.




Exercício a)

In [ ]:
import re
import json

def split_first_last_name(name):
    return re.match(r"\w+\b", name).group(), re.search(r"\b\w+$", name).group()

def parse(path):
  file = open(path)
  regex_str = r"(?P<dir>\d+)::(?P<year>\d{4})-(?P<month>\d{2})-(?P<day>\d{2})::(?P<name>[a-zA-Z ]+)::(?P<father>[a-zA-Z ]+)::(?P<mother>[a-zA-Z ]+)::(?P<obs>[^:]*)::"
  res = []
  regex = re.compile(regex_str)
  for line in file.readlines():
    if match := regex.finditer(line):
      res = res + [m.groupdict() for m in match]

  return res

data = parse("processos.txt")

def group_by_year(data):
  res = {}
  for entry in data:
    if entry["year"] not in res:
      res[entry["year"]] = 0
    
    res[entry["year"]] += 1
  
  return res

print(group_by_year(data))

Exercício b)

In [ ]:
def top5_names_freq(data,century, idx):
  names = {}
  
  for entry in data:
    if (int(entry["year"]) - 1) // 100 + 1 == century:
      name = split_first_last_name(entry["name"])
      if name[idx] not in names:
        names[name[idx]] = 0

      names[name[idx]] += 1

      father = split_first_last_name(entry["father"])
      if father[idx] not in names:
        names[father[idx]] = 0

      names[father[idx]] += 1

      mother = split_first_last_name(entry["mother"])
      if mother[idx] not in names:
          names[mother[idx]] = 0

      names[mother[idx]] += 1
  res = sorted(names.items(), key=lambda x:x[1], reverse=True)[:5]
  return res

def top5_names_freq_century(data):
  first_name = {}
  last_name = {}
  for i in range(0,22):
    first_name[i] = top5_names_freq(data,i,0)
    last_name[i] = top5_names_freq(data,i,1)

  return first_name, last_name

def top5_names_freq_all(data):
  first_name, last_name  = top5_names_freq_century(data)
  sum_first_name = {}
  sum_last_name = {}
  aux = []

  for lst in first_name.values():
    aux.extend(lst)
    for k, v in lst:
      if k not in sum_first_name:
        sum_first_name[k] = v
      else:
        sum_first_name[k] += v
    
  aux.clear()

  for lst in last_name.values():
    aux.extend(lst)
    for k, v in lst:
      if k not in sum_last_name:
        sum_last_name[k] = v
      else:
        sum_last_name[k] += v

  f_names = sorted(sum_first_name.items(), key=lambda x:x[1], reverse=True)[:5]
  l_names = sorted(sum_last_name.items(), key=lambda x:x[1], reverse=True)[:5]
  return f_names, l_names
  

print(top5_names_freq_century(data))
print(top5_names_freq_all(data))

Exercício c)

In [ ]:
def freq_relation(data):
  pattern = re.compile(r",[^\s*\d+][\w\s]*.[\s*](?i:Proc.)")
  res = {}
  for entry in data:
    if match := pattern.search(entry["obs"]):
      match = match.group().lower()
      match = match[1:-7]
      if match not in res:
        res[match] = 0

      res[match.lower()] += 1

  return res

print(freq_relation(data))

Exercício d)

In [ ]:
def to_json(data, out_file):
  if ".json" not in out_file:
    out_file = out_file + ".json"

  file = open(out_file, "w")
  file.write("[\n")
  for i in range(0,20):
    json.dump(data[i], file, indent=4, separators=(',', ': '))  
    if i != 19: file.write(",\n")
  file.write("\n]")

  file.close()

to_json(data,"teste")